In [ ]:
import os

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
try:
    import open3d as o3d
except ImportError:
    import sys
    print(sys.exc_info())
import pandas as pd
from scipy import interpolate

from dosipy.utils.integrate import elementwise_dblquad
from dosipy.utils.viz import set_axes_equal, scatter_2d, scatter_3d

from utils import load_processed_surface_data
from utils import export_rect_idx
from utils import diff

In [ ]:
# CST data

path_results = os.path.join('data', 'results')
data = [['Dipole', 'Vertical', 5,
         [-0.009196137125833444, -0.008943818176342983], 2.97],
        ['Dipole', 'Vertical', 10,
         [-0.01052339327664423, -0.010288400579897647], 2.13],
        ['Dipole', 'Vertical', 15,
         [-0.01052339327664423, -0.010288400579897647], 1.45],
        ['Array', 'Vertical', 5,
         [-0.010974217437555025, -0.0110377357372408], 5.62],
        ['Array', 'Vertical', 10,
         [-0.008974217437555027, -0.009037735737240808], 5.92],
        ['Array', 'Vertical', 15,
         [-0.008974686708799248, -0.009037296281864424], 5.23],
        ['Dipole', 'Horizontal', 5,
         [-0.008307550770888367, -0.00837106907057417], 3.07],
        ['Dipole', 'Horizontal', 10,
         [-0.008316858065744332, -0.00835886736452144], 1.92],
        ['Dipole', 'Horizontal', 15,
         [-0.008316834501406759, -0.008358509577943802], 1.29],
        ['Array', 'Horizontal', 5,
         [-0.010307550770888359, -0.01037106907057414], 5.77],
        ['Array', 'Horizontal', 10,
         [-0.00831685806574433, -0.00835886736452144], 6.07],
        ['Array', 'Horizontal', 15,
         [-0.008316858065744323, -0.00835886736452144], 5.96]]
ds = pd.DataFrame(data, columns=['antenna', 'orientation', 'distance',
                                 'origin', 'APD_CST'])
# ds.to_csv(os.path.join(path_results, f'Results_APD.csv'))

In [ ]:
ds

In [ ]:
# extract data

i = 0
antenna = ds.loc[i, 'antenna']
orientation = ds.loc[i, 'orientation']
distance = ds.loc[i, 'distance']
origin = ds.loc[i, 'origin']

df = load_processed_data(f'{antenna}{orientation}', distance)
xyz = df[['x', 'y', 'z']].values
PD = df['PD'].values
Sx = df['Sx'].astype('complex128').values
Sy = df['Sy'].astype('complex128').values
Sz = df['Sz'].astype('complex128').values

fig, ax = scatter_3d({'x': xyz[:, 0],
                      'y': xyz[:, 1],
                      'z': xyz[:, 2],
                      'PD': PD}, elev=[20], azim=[160])
ax.set_box_aspect([1, 1, 1])
ax = set_axes_equal(ax)
ax.set_title(f'{antenna}{orientation} at {distance} mm');

In [ ]:
# extract points visible from the EMF incidence POV (x direction)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
diameter = np.linalg.norm(
    pcd.get_max_bound() - pcd.get_min_bound()
)
radius = 100
center = pcd.get_center()
camera = [center[0] - diameter, center[1], center[2]]
_, mask = pcd.hidden_point_removal(camera, radius)
xyz_yz = xyz[mask]
PD_yz = PD[mask]
Sx_yz = Sx[mask]
Sy_yz = Sy[mask]
Sz_yz = Sz[mask]

fig, ax = scatter_3d({'x': xyz_yz[:, 0],
                      'y': xyz_yz[:, 1],
                      'z': xyz_yz[:, 2],
                      'PD': PD_yz}, elev=[10], azim=[180])
ax.set_box_aspect([1, 1, 1])
ax = set_axes_equal(ax)
ax.set_title(f'{antenna}{orientation} at {distance} mm');

In [ ]:
# find an appropriate domain

edge_length = 0.02
area = edge_length ** 2
avg_center = [origin[0] + edge_length / 2,
              origin[1] + edge_length / 2]
_, rect_idx = export_rect_idx(xyz=xyz_yz,
                              center=avg_center,
                              edge_length=edge_length,
                              view='yz')

fig, ax = scatter_2d({'y': xyz_yz[:, 1],
                      'z': xyz_yz[:, 2],
                      'PD': PD_yz}, s=0.5)
ax.plot(*avg_center, 'x', ms=10, c='r')
patch_rect = Rectangle(origin, edge_length, edge_length,
                       fc='none', ec='r', lw=2)
ax.add_patch(patch_rect)
ax.invert_xaxis()

In [ ]:
# functional reconstruction of the integration surface

xyz_int = xyz_yz[rect_idx]  # integration surface
Sx_int = Sx_yz[rect_idx]
Sy_int = Sy_yz[rect_idx]
Sz_int = Sz_yz[rect_idx]
x_fun = interpolate.RBFInterpolator(xyz_int[:, 1:],
                                    xyz_int[:, 0],
                                    neighbors=333)
x_int_approx = x_fun(xyz_int[:, 1:])
np.all(np.isclose(xyz_int[:, 0], x_int_approx))

In [ ]:
# normals and their associated lengths on the integration surface

pcd_int = o3d.geometry.PointCloud()
pcd_int.points = o3d.utility.Vector3dVector(xyz_int)
pcd_int.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(33))
pcd_int.orient_normals_consistent_tangent_plane(33)
n_int = np.asarray(pcd_int.normals)

dxdy = diff(x_fun, arg=0)
dxdz = diff(x_fun, arg=1)
n_len = np.sqrt(dxdy(xyz_int[:, 1:]) ** 2
                + dxdz(xyz_int[:, 1:]) ** 2
                + 1)

In [ ]:
# averaging and normalization with fixed area of 4 cm2

pdn_int = np.abs(Sx_int.real * n_int[:, 0]
                 + Sy_int.real * n_int[:, 1]
                 + Sz_int.real * n_int[:, 2]) * n_len

apd = elementwise_dblquad(xyz_int[:, 1:],
                          values=pdn_int,
                          degree=51,
                          interp_func=interpolate.LinearNDInterpolator,
                          fill_value=0
                          ) / area
apd